1. Выберите на сайте объявлений интересующую вас рубрику самого нижнего уровня (например, Для дома и дачи – Мебель и интерьер – Кровати, диваны и кресла – Диваны).

2. Убедитесь в том, что сайт учел в выдаче ваше местоположение (если нет – укажите его).

3. Обратите внимание на адрес полученной вами страницы и на то, как он изменяется при переходе к следующей странице выдачи (где и как указывается номер страницы), зафиксируйте это.

4. Откройте код страницы выдачи (F12 или правой кнопкой мыши). Выберите ключевые элементы блока записи об одном объявлении так, чтобы вы могли выделить этот блок из всего текста: например, это может быть определенный тег, с определенным набором атрибутов.

5. Напишите скрипт, который перебирает все страницы выдачи, и собирает в список следующие параметры: id объявления, название, цена, срок публикации.

6. Проверьте полученный список на корректность – в нем не должно быть посторонних записей и выбросов (слишком малых, слишком больших и отсутствующих цен). При необходимости, устраните ненужные записи программно.

7. Отфильтруйте полученный список по заданному вами критерию: например, оставьте только цены между заданными вами значениями, или оставьте только объявления, находящие в просмотре не больше суток.

8. Сохраните результат вашей работы так, чтобы он был пригоден для машинного анализа.

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np

def print_(data):
    for x in data:
        print(x)
    print(len(data))
#получение информации
def get_inform():
    url = 'https://youla.ru/belgorod/ehlektronika/televizory-proektory'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    items = soup.find_all('li', class_ = 'product_item')
    data = []
    for item in items:
        data.append(
            {
                'id': item['data-id'],
                'title': item.find('div', class_ = 'product_item__title').text,
                #берем значение текст из тега, ищем числа, из списка делаем строку, а из нее число
                'description': int(''.join(re.findall('[0-9]+', item.find('div', class_ = 'product_item__description').find('div').text))),
                'date': item.find('div', class_ = 'product_item__date').find('span', class_ = 'visible-xs').text
            }
        )
    return data
#проверка на телевизоры
def correctness(data):
    for item in data:
        if not re.search('телевизор.+', item['title'], re.I): 
            #print(item)
            data.remove(item)
    #print_(data)

def find_anomalies(elem):
    data = [x['description'] for x in elem]
    anomalies = []
    data_std = np.std(data) #среднеквадратичное отклонение
    data_mean = np.mean(data) #среднее значение
    anomaly_cut_off = data_std * 2.5 #больше-меньше в 2.5 раза от среднеквадратичного отклонения
    #нижний и верхний предел 
    lower_limit  = data_mean - anomaly_cut_off 
    upper_limit = data_mean + anomaly_cut_off
    # проверка вхождений в этот диапазон
    for i, outlier in enumerate (data):
        if outlier > upper_limit or outlier < lower_limit:
            anomalies.append([i,outlier])
    #удаление тех, что не входят 
    for x in anomalies:
        elem.pop(x[0])
    #print_(elem)

def sort_data(data):
    #диапазон больше-меньше
    sort_=[]
    for i, item in enumerate (data):
        if item['description'] > 18_000 or item['description'] < 5_000:
            sort_.append([i,item])
    sort_.sort(reverse=True)
    for x in sort_:
        data.pop(x[0])
    #print_(data)


data = get_inform()
print_(data)

print('\n\n')
correctness(data)
find_anomalies(data)
sort_data(data)
print_(data)

#запись в csv файл
FILENAME = "data.csv"
with open(FILENAME, "w", newline="") as file:
    columns = data[0].keys()
    writer = csv.DictWriter(file, fieldnames=columns)
    writer.writeheader()
    writer.writerows(data)

{'id': '606d0c3420098e6d25288a6d', 'title': 'Телевизоры и плазменные панели Polarline 32PL13TC', 'description': 9490, 'date': '07.04.2021'}
{'id': '606d0c2948aada245777254d', 'title': 'Телевизоры и плазменные панели Sony KD-43XH8096', 'description': 45990, 'date': '07.04.2021'}
{'id': '606d0c2e447f9d4b40115d8a', 'title': 'Телевизоры и плазменные панели Samsung QE32LS03TBK', 'description': 25990, 'date': '07.04.2021'}
{'id': '606d0c3885af6550bb3dc564', 'title': 'Телевизоры и плазменные панели Samsung LE19B450C4W', 'description': 3490, 'date': '07.04.2021'}
{'id': '606cdc795087277e4c00b293', 'title': 'Телевизор SHIVAKI', 'description': 1800, 'date': '07.04.2021'}
{'id': '5fbcba3b9265cc00b54b926f', 'title': 'Телевизор LG CINEMA 3D 32”', 'description': 12000, 'date': '04.04.2021'}
{'id': '606c6c27e90cf36261407d5b', 'title': 'Гигант 4К телевизор Sony 165cm.65дюймов', 'description': 55500, 'date': '06.04.2021'}
{'id': '5a900737f235027bad640d05', 'title': 'Телевизор Philips 52 см', 'descripti